In [1]:
import pickle

from itertools import product
import sys
sys.path.append("../src")
from denoising_methods import *

# Load data

In [3]:
with open("../data/eeg_datasets.pkl", "rb") as f:
    eeg_datasets_dict = pickle.load(f)

original_datasets = eeg_datasets_dict["datasets"]
noisy_datasets = eeg_datasets_dict["datasets_artnoise"]

# Perform denoising - artificial noise scenario

### Wavelet Transform with Thresholding

#### Preparing parameters

In [4]:
wavelet_types = ['haar', 'coif4', 'coif8', 'db4', 'db8', 'sym4', 'sym8']
levels = [1, 2, 3, 4, 5]
threshold_modes = ['soft', 'hard']

param_combinations = product(wavelet_types, levels, threshold_modes)

In [5]:
results_wt = []

In [6]:
for original, noisy in zip(original_datasets, noisy_datasets):
    signal, times = noisy["noisy_data"], original["times"]
    denoised_dict = {
        "dataset_name": original["name"],
        "parameter_sets": [],
        "denoised_signals": [],
    }
    param_combinations = product(wavelet_types, levels, threshold_modes)
    for wavelet_type, level, threshold_mode in param_combinations:
        params = {
            "wavelet_type": wavelet_type,
            "level": level,
            "threshold_mode": threshold_mode
        }
        denoised_dict["parameter_sets"].append(params)
        denoised_signal = wt_denoise_multichannel(signal, wavelet_type, level, threshold_mode)
        # print(denoised_signal.shape, signal.shape)
        denoised_dict["denoised_signals"].append(denoised_signal)
    results_wt.append(denoised_dict)


### ICA

In [7]:
n_components_list = [None, 5, 10, 15, 20]

In [8]:
results_ica = []

In [9]:
for original, noisy in zip(original_datasets, noisy_datasets):
    signal, times = noisy["noisy_data"], original["times"]
    
    denoised_dict = {
        "dataset_name": original["name"],
        "parameter_sets": [],
        "denoised_signals": [],
    }

    for n_components in n_components_list:
        params = {
            "n_components": n_components,
        }
        denoised_dict["parameter_sets"].append(params)
        
        denoised_signal = ica_denoise_multichannel(signal, n_components=n_components)
        denoised_dict["denoised_signals"].append(denoised_signal)

    results_ica.append(denoised_dict)

e:\anaconda3\envs\eeg_denoising\Lib\site-packages\sklearn\decomposition\_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
e:\anaconda3\envs\eeg_denoising\Lib\site-packages\sklearn\decomposition\_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
e:\anaconda3\envs\eeg_denoising\Lib\site-packages\sklearn\decomposition\_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


# Perform denoising - original data scenario

### Wavelet Transform with Thresholding

In [10]:
results_wt_orig = []

In [12]:
wavelet_types = ['coif4', 'coif8', 'db8', 'sym8']
levels = [3,]
threshold_modes = ['soft', 'hard']

param_combinations = product(wavelet_types, levels, threshold_modes)

In [13]:
for original in original_datasets:
    signal, times = original['data'], original['times']
    denoised_dict = {
        "dataset_name": original['name'],
        "parameter_sets": [],
        "denoised_signals": [],
    }
    param_combinations = product(wavelet_types, levels, threshold_modes)
    for wavelet_type, level, threshold_mode in param_combinations:
        params = {
            "wavelet_type": wavelet_type,
            "level": level,
            "threshold_mode": threshold_mode
        }
        denoised_dict["parameter_sets"].append(params)
        denoised_signal = wt_denoise_multichannel(signal, wavelet_type, level, threshold_mode)
        denoised_dict["denoised_signals"].append(denoised_signal)
    results_wt_orig.append(denoised_dict)

### ICA

In [14]:
n_components_list = [1,2,3,4,5,15,]

In [15]:
results_ica_orig = []

In [16]:
for original in original_datasets:
    signal, times = original['data'], original['times']
    
    denoised_dict = {
        "dataset_name": original['name'],
        "parameter_sets": [],
        "denoised_signals": [],
    }

    for n_components in n_components_list:
        params = {
            "n_components": n_components,
        }
        denoised_dict["parameter_sets"].append(params)

        denoised_signal = ica_denoise_multichannel(signal, n_components=n_components)
        denoised_dict["denoised_signals"].append(denoised_signal)

    results_ica_orig.append(denoised_dict)

# Save data to pkl

In [18]:
len(results_wt_orig), len(results_wt), len(results_ica), len(results_ica_orig)

(1, 1, 1, 1)

In [19]:
with open("../data/denoised_data.pkl", "wb") as f:
    pickle.dump(
        {"results_wavelet_denoising_artnoise": results_wt, 
         "results_wavelet_denoising_original": results_wt_orig,
         "results_ica_artnoise": results_ica,
         "results_ica_original": results_ica_orig}, f)